In [3]:
import cv2
import os
data=[]
target=[]
data_path=r"C:\Users\ASUS\Osteoporosis_Knee_X-ray"
categories=['osteopenia','normal','osteoporosis']
map={0:'osteopenia',1:'normal',2:'osteoporosis'}
labels=[0,1,2]
idd=[]
label_dict=dict(zip(categories,labels))
def label(s):
    if 'OP' in s:
        return 0 
    if 'OS' in s:      
        return 2 
    if 'N' in s: 
        return 1 
print(label_dict)
print(categories)
print(labels)
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        try:
            #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            resized=cv2.resize(img,(224, 224))
            data.append(resized)
            target.append(label(img_name))
        except Exception as e:
            print('Exception:',e)
# print(data)

{'osteopenia': 0, 'normal': 1, 'osteoporosis': 2}
['osteopenia', 'normal', 'osteoporosis']
[0, 1, 2]


In [25]:
import pandas as pd
df=pd.read_excel(r'patient_details_updated.xlsx')
# df
# df.to_excel(r'C:\Users\vamsi\Downloads\Osteoporosis Knee X-ray\patient_details_updated.xlsx',index=False)

In [26]:
import numpy as np 
np.save('data.npy',data)
data2=np.load('data.npy')
data2.shape

(239, 224, 224, 3)

In [27]:
df.columns

Index(['S.No', 'Patient Id', 'Joint Pain:', 'Gender', 'Age', 'Menopause Age',
       'height  (meter)', 'Weight (KG) ', 'Smoker', 'Alcoholic', 'Diabetic',
       'Hypothyroidism', 'Number of Pregnancies', 'Seizer Disorder',
       'Estrogen Use', 'Occupation ', 'History of Fracture', 'Dialysis:',
       'Family History of Osteoporosis', 'Maximum Walking distance (km)',
       'Daily Eating habits', 'Medical History', 'T-score Value',
       'Z-Score Value', 'BMI: ', 'Site', 'Obesity', 'Diagnosis'],
      dtype='object')

In [28]:
import pandas  as pd
df=pd.read_csv(r"updated_data.csv")
# df.head(1)
df=df.rename(columns={'data':'images'})
# df.columns

In [29]:

from sklearn.preprocessing import StandardScaler
# Selecting only the important columns
important_columns = ['S.No', 'Patient Id', 'Joint Pain:', 'Gender', 'Age', 'Menopause Age',
                     'height  (meter)', 'Weight (KG) ', 'Smoker', 'Alcoholic', 'Diabetic',
                     'Hypothyroidism', 'Number of Pregnancies', 'Seizer Disorder',
                     'Estrogen Use', 'Occupation ', 'History of Fracture', 'Dialysis:',
                     'Family History of Osteoporosis', 'Maximum Walking distance (km)',
                     'Daily Eating habits', 'Medical History', 'T-score Value',
                     'Z-Score Value', 'BMI: ', 'Site', 'Obesity', 'Diagnosis', 
                     ]

df = df[important_columns]

# Assuming you want to scale 'Age', 'Height (meter)', and 'Weight (KG)'
columns_to_scale = ['Age', 'Menopause Age', 'height  (meter)', 'Weight (KG) ', 'Number of Pregnancies',
                    'Maximum Walking distance (km)', 'T-score Value', 'Z-Score Value', 'BMI: ']

# Apply scaling to the selected columns
scaler = StandardScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

# Now you have only the important columns and necessary columns are scaled and standardized
df.head()


,S.No,Patient Id,Joint Pain:,Gender,Age,Menopause Age,height (meter),Weight (KG),Smoker,Alcoholic,...,Family History of Osteoporosis,Maximum Walking distance (km),Daily Eating habits,Medical History,T-score Value,Z-Score Value,BMI:,Site,Obesity,Diagnosis
0,1.0,OP1,yes,male,1.103660,NaN,0.631320,-0.423468,yes,no,...,no,-0.871801,normal,normal,-0.207519,3.055871,-0.891038,knee,normal weight,osteopenia
1,2.0,OP2,yes,female,0.230497,0.189324,0.949663,0.934339,no,no,...,no,-0.468029,normal,"G.B rem, B.P",-0.617049,-2.046204,0.039387,knee,over weight,osteopenia
2,3.0,OP3,yes,female,-0.166396,-0.418957,-2.233767,-1.050148,no,no,...,no,-0.468029,low protiens,no,-0.119762,-0.266410,0.931567,knee,obesity,osteopenia
3,4.0,OP4,yes,female,0.468632,-1.027238,-0.642052,0.307659,no,no,...,no,0.036687,low fats,uterus removed,-0.573170,0.297191,0.842154,knee,obesity,osteopenia
4,5.0,OP5,yes,female,-0.880803,NaN,-0.960395,-1.572382,no,no,...,no,0.036687,normal,depression,0.041124,0.415844,-0.837833,knee,normal weight,osteopenia


In [31]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Assuming df is your DataFrame with necessary columns

# Columns to be one-hot encoded
columns_to_one_hot_encode = ['Joint Pain:', 'Gender', 'Smoker', 'Alcoholic', 'Diabetic', 'Hypothyroidism', 'Seizer Disorder', 
                             'Estrogen Use', 'Occupation ', 'History of Fracture', 'Dialysis:', 
                             'Family History of Osteoporosis', 'Daily Eating habits', 'Medical History', 'Obesity', 'Diagnosis']

# Columns to be label encoded
columns_to_label_encode = ['Site']

# One-hot encoding
encoded_features = []
for column in columns_to_one_hot_encode:
    one_hot_encoder = OneHotEncoder(sparse=False)
    encoded_column = one_hot_encoder.fit_transform(df[[column]])
    encoded_features.append(pd.DataFrame(encoded_column, columns=one_hot_encoder.get_feature_names([column])))
    
# Concatenate one-hot encoded features
df_encoded = pd.concat([df] + encoded_features, axis=1)

# Label encoding
for column in columns_to_label_encode:
    label_encoder = LabelEncoder()
    df_encoded[column] = label_encoder.fit_transform(df_encoded[column])

# Now you have encoded the categorical columns using one-hot encoding and label encoding
df_encoded.head()
print(len(df.columns))

28


In [ ]:
# a=[]
# a.append(df.columns)
# print(a)
# print(len(a[0]))

In [32]:
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame with necessary columns

# Columns to be label encoded
columns_to_label_encode = ['Joint Pain:', 'Gender', 'Smoker', 'Alcoholic', 'Diabetic', 'Hypothyroidism', 'Seizer Disorder', 
                             'Estrogen Use', 'Occupation ', 'History of Fracture', 'Dialysis:', 
                             'Family History of Osteoporosis', 'Daily Eating habits', 'Medical History', 'Obesity', 'Diagnosis', 'Site']

# Label encoding
label_encoders = {}
for column in columns_to_label_encode:
    label_encoder = LabelEncoder()
    df[column] = label_encoder.fit_transform(df[column])
    label_encoders[column] = label_encoder

# Now you have encoded the categorical columns using label encoding
print(df.head())


   S.No Patient Id  Joint Pain:  Gender       Age  Menopause Age  \
0   1.0        OP1            1       1  1.103660            NaN   
1   2.0        OP2            1       0  0.230497       0.189324   
2   3.0        OP3            1       0 -0.166396      -0.418957   
3   4.0        OP4            1       0  0.468632      -1.027238   
4   5.0        OP5            1       0 -0.880803            NaN   

   height  (meter)  Weight (KG)   Smoker  Alcoholic  ...  \
0         0.631320     -0.423468       1          0  ...   
1         0.949663      0.934339       0          0  ...   
2        -2.233767     -1.050148       0          0  ...   
3        -0.642052      0.307659       0          0  ...   
4        -0.960395     -1.572382       0          0  ...   

   Family History of Osteoporosis  Maximum Walking distance (km)  \
0                               0                      -0.871801   
1                               0                      -0.468029   
2                         

In [ ]:
# df['images']

In [33]:
df=df.fillna(0)

In [34]:
X=df.drop(columns=['Diagnosis','Patient Id'])
y=df['Diagnosis']
# print(X['images'].head())


In [ ]:
# import pandas as pd
# import numpy as np

# # Define the extract_features function
# def extract_features(image):
#     # Calculate mean pixel intensity for each channel
#     mean_intensity = np.mean(image, axis=(0, 1))
#     # Calculate image dimensions
#     dimensions = image.shape[:2]  # Assuming images are grayscale
    
#     return pd.Series(np.concatenate([mean_intensity, dimensions]))

# # Apply the extract_features function to each image in the images column
# X[['mean_intensity_1', 'mean_intensity_2', 'mean_intensity_3', 'image_height', 'image_width']] = data.apply(extract_features)

# # Drop the original 'images' column
# # X = X.drop(columns=['images'])

# # Convert DataFrame columns to float32
# X = X.astype('float32')
# y = y.astype('float32')


In [35]:

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Assuming df is your DataFrame with preprocessed data

# Separate features (X) and target variable (y)
X = df.drop(columns=['Diagnosis','Patient Id'])  # Replace 'Target_column' with the name of your target column
y = df['Diagnosis']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# # Print DataFrame info to check data types and missing values
# print(X.info())
# print(y.info())

In [36]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model with Adam optimizer and binary cross-entropy loss
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train_scaled, y_train, epochs=30, batch_size=28, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Epoch 1/30
6/6 [==============================] - 1s 36ms/step - loss: 0.7722 - accuracy: 0.3026 - val_loss: 0.6498 - val_accuracy: 0.3846
Epoch 2/30
6/6 [==============================] - 0s 7ms/step - loss: 0.5345 - accuracy: 0.4737 - val_loss: 0.4430 - val_accuracy: 0.5641
Epoch 3/30
6/6 [==============================] - 0s 7ms/step - loss: 0.3545 - accuracy: 0.5921 - val_loss: 0.2747 - val_accuracy: 0.6410
Epoch 4/30
6/6 [==============================] - 0s 7ms/step - loss: 0.1898 - accuracy: 0.6316 - val_loss: 0.1350 - val_accuracy: 0.6923
Epoch 5/30
6/6 [==============================] - 0s 7ms/step - loss: 0.0471 - accuracy: 0.6316 - val_loss: 0.0064 - val_accuracy: 0.6923
Epoch 6/30
6/6 [==============================] - 0s 7ms/step - loss: -0.0865 - accuracy: 0.6382 - val_loss: -0.1152 - val_accuracy: 0.6667
Epoch 7/30
6/6 [==============================] - 0s 7ms/step - loss: -0.2187 - accuracy: 0.6382 - val_loss: -0.2332 - val_accuracy: 0.6667
Epoch 8/30
6/6 [=============

DENSENET

In [5]:
import numpy as np
# y=np.array(y,dtype=float)
# np.save('target.npy',y)
y=np.load('target.npy')
data2=np.load('data2.npy')

In [6]:
from tensorflow.keras.utils import to_categorical
data2= data2.astype('float32') / 255.0
labels = to_categorical(y)
len(labels)

D:\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


239

In [7]:
from sklearn.model_selection import train_test_split
train_data, val_data, train_labels, val_labels = train_test_split(data2, labels, test_size=0.2, random_state=42)

In [8]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
print(train_data.shape)
print(train_labels.shape)

(191, 224, 224, 3)
(191, 3)


In [11]:
history = model.fit(train_data, train_labels, batch_size=32, epochs=5, validation_data=(val_data, val_labels))

Epoch 1/5
6/6 [==============================] - 111s 18s/step - loss: 0.4619 - accuracy: 0.8586 - val_loss: 1.0715 - val_accuracy: 0.4792
Epoch 2/5
6/6 [==============================] - 104s 17s/step - loss: 0.2796 - accuracy: 0.9267 - val_loss: 1.0064 - val_accuracy: 0.5625
Epoch 3/5
6/6 [==============================] - 109s 18s/step - loss: 0.1741 - accuracy: 0.9476 - val_loss: 0.9457 - val_accuracy: 0.6042
Epoch 4/5
6/6 [==============================] - 124s 20s/step - loss: 0.1545 - accuracy: 0.9476 - val_loss: 0.9529 - val_accuracy: 0.6667
Epoch 5/5
6/6 [==============================] - 103s 17s/step - loss: 0.1031 - accuracy: 0.9476 - val_loss: 1.0149 - val_accuracy: 0.6875


In [13]:
predictions=model.predict(train_data[2])
predictions

ValueError: in user code:

    File "D:\anaconda\lib\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "D:\anaconda\lib\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\anaconda\lib\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "D:\anaconda\lib\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "D:\anaconda\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "D:\anaconda\lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "model" is incompatible with the layer: 

In [1]:
pip show keras


Name: keras
Version: 3.3.3
Summary: Multi-backend Keras.
Note: you may need to restart the kernel to use updated packages.


Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: d:\anaconda\lib\site-packages
Requires: namex, h5py, ml-dtypes, numpy, optree, rich, absl-py
Required-by: tensorflow-intel


In [ ]:
pip install keras==<2.13.1>
